<a href="https://colab.research.google.com/github/Hashhhhhhhh/Quantum-Computing-using-Qiskit/blob/main/QSVM_example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
# Number of qubits
n_qubits = 3
dim = 2 ** n_qubits

In [19]:
#single-qubit rotation around Y-axis
def RY(theta):
    return np.array([[np.cos(theta / 2), -np.sin(theta / 2)],
                     [np.sin(theta / 2),  np.cos(theta / 2)]])

In [20]:
# Define single-qubit Identity
I = np.eye(2)

# Construct multi-qubit gate from single-qubit gates via tensor product
def kron_n(gates):
    result = np.array([[1]])
    for g in gates:
        result = np.kron(result, g)
    return result

In [21]:
# Construct CNOT gate on 3 qubits
def CNOT(control, target, n_qubits=3):
    size = 2 ** n_qubits
    mat = np.zeros((size, size), dtype=complex)
    for i in range(size):
        bin_str = np.binary_repr(i, width=n_qubits)
        bits = list(map(int, bin_str))
        if bits[control] == 0:
            j = i
        else:
            bits[target] ^= 1  # flip target bit
            j = int("".join(map(str, bits)), 2)
        mat[j, i] = 1
    return mat

In [22]:
# Apply a single-qubit gate to qubit q in n_qubits system
def apply_single_qubit_gate(gate, q, n_qubits=3):
    gates = []
    for i in range(n_qubits):
        if i == q:
            gates.append(gate)
        else:
            gates.append(I)
    return kron_n(gates)

In [23]:
# Feature map as matrix
def feature_map_matrix(x):
    U = np.eye(dim, dtype=complex)
    # Step 1: RY rotations by x_i * pi
    for q in range(n_qubits):
        U = apply_single_qubit_gate(RY(x[q] * np.pi), q) @ U
    # Step 2: CNOT(0->1) then CNOT(1->2)
    U = CNOT(0, 1) @ U
    U = CNOT(1, 2) @ U
    # Step 3: RY rotations by x_i * pi/2
    for q in range(n_qubits):
        U = apply_single_qubit_gate(RY(x[q] * np.pi / 2), q) @ U
    return U

In [24]:
# Prepare |0...0> state
zero_state = np.zeros(dim, dtype=complex)
zero_state[0] = 1

# Quantum kernel between x and y
def quantum_kernel(x, y):
    Ux = feature_map_matrix(x)
    Uy = feature_map_matrix(y)
    # Apply U_y^\dagger U_x to |0>
    psi = Uy.conj().T @ Ux @ zero_state
    # Kernel = probability amplitude squared of |000>
    return np.abs(psi[0]) ** 2

In [25]:
# Construct kernel matrix for dataset X
def construct_kernel_matrix(X1, X2=None):
    if X2 is None:
        X2 = X1
    n1 = len(X1)
    n2 = len(X2)
    K = np.zeros((n1, n2))
    for i in range(n1):
        for j in range(n2):
            K[i, j] = quantum_kernel(X1[i], X2[j])
    return K

In [26]:
X = np.array([
    [0.1, 0.3, 0.5],
    [0.2, 0.1, 0.4],
    [0.4, 0.7, 0.2],
    [0.9, 0.6, 0.8],
    [0.7, 0.9, 0.3]
])

K = construct_kernel_matrix(X)
print("Quantum kernel matrix:\n", K)

Quantum kernel matrix:
 [[1.         0.72118853 0.13412747 0.05070486 0.03030693]
 [0.72118853 1.         0.05199311 0.00710148 0.01818903]
 [0.13412747 0.05199311 1.         0.01881182 0.53040472]
 [0.05070486 0.00710148 0.01881182 1.         0.20259647]
 [0.03030693 0.01818903 0.53040472 0.20259647 1.        ]]


In [27]:
from sklearn.svm import SVC

# Assuming you already have the kernel matrix K and labels y
# Example labels: binary classification
y = np.array([0, 0, 1, 1, 1])

# Train classical SVM using precomputed quantum kernel matrix
svm = SVC(kernel='precomputed')
svm.fit(K, y)

# Classify a new sample using the quantum kernel with training data
def classify_new(sample, X_train, svm_model):
    # Compute kernel vector between sample and training data
    kernel_vec = np.array([quantum_kernel(sample, x) for x in X_train]).reshape(1, -1)
    # Predict class label
    return svm_model.predict(kernel_vec)[0]

# Test classification on a new data point
new_sample = np.array([0.3, 0.5, 0.4])
predicted_label = classify_new(new_sample, X, svm)
print("Predicted class for new sample:", predicted_label)
print()
print(K)

Predicted class for new sample: 0

[[1.         0.72118853 0.13412747 0.05070486 0.03030693]
 [0.72118853 1.         0.05199311 0.00710148 0.01818903]
 [0.13412747 0.05199311 1.         0.01881182 0.53040472]
 [0.05070486 0.00710148 0.01881182 1.         0.20259647]
 [0.03030693 0.01818903 0.53040472 0.20259647 1.        ]]
